In [1]:
from selenium.webdriver.common.keys import Keys 
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs 
from datetime import datetime, timedelta 
import time 
import pandas as pd 
import re 
import zipfile 
import os 
import requests 
from tqdm import tqdm


In [2]:
webdriver_path = os.getcwd()+'.//webdriver//chromedriver.exe'

In [4]:
url_base = 'https://www.koreabaseball.com/Record/Player/{category}' 
category_list = ['HitterBasic/Basic1.aspx', 'PitcherBasic/Basic1.aspx', 'Defense/Basic.aspx', 'Runner/Basic.aspx'] 
driver = wd.Chrome(executable_path='.\\webdriver\\chromedriver.exe')
driver.implicitly_wait(3)

<ipython-input-4-09dbd55a7047>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wd.Chrome(executable_path='.\\webdriver\\chromedriver.exe')


In [5]:
driver.get('https://www.koreabaseball.com/Record/Player/Defense/Basic.aspx')

In [13]:
driver.get('https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx?sort=HRA_RT')

In [14]:
driver.close()

In [21]:
from selenium import webdriver 
import time 
import pandas as pd 

url_base = 'https://www.koreabaseball.com/Record/Player/{category}' 
category_list = ['HitterBasic/Basic1.aspx', 'PitcherBasic/Basic1.aspx', 'Defense/Basic.aspx', 'Runner/Basic.aspx'] 
driver = webdriver.Chrome(webdriver_path) 
driver.implicitly_wait(3) 

from tqdm import tqdm 
baseball_data = [] 

for category in category_list: 
    url = url_base.format(category = category) 
    driver.get(url) 
    driver.maximize_window() 
    year_list = [x.strip() for x in driver.find_element_by_class_name('select03').text.split('\n') if len(x) >1 ] 
    category_data_list = [] 
    for year in tqdm(year_list[:3]): 
        driver.find_element_by_class_name('select03').click() 
        xpath_format = '//option[@value="{year}"]' 
        driver.find_element_by_xpath(xpath_format.format(year = year)).click() 
        time.sleep(0.1) 
        temp_list = [x.split(' ') for x in driver.find_element_by_tag_name('table').text.split('\n')] 
        temp_data = pd.DataFrame(columns = temp_list[0]) 
        for i,temp in enumerate(temp_list[1:]): 
            try: 
                temp_data.loc[i] = temp 
                time.sleep(0.5) 
            except: 
                ip_index = temp_list[0].index('IP') 
                ip = temp[ip_index] 
                del temp[13] 
                temp[13] = ip+' '+temp[13] 
                
                temp_data.loc[i] = temp 
                time.sleep(0.5) 
                
        temp_data['Year'] = year 
        category_data_list.append(temp_data) 
        time.sleep(0.1) 
        time.sleep(1) 
    category_data = pd.concat(category_data_list) 
    baseball_data.append(category_data)

<ipython-input-21-bd78ff6b9f1a>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(webdriver_path)
C:\Users\SEC\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:630: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
  0%|                                                                                            | 0/3 [00:00<?, ?it/s]C:\Users\SEC\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:430: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
C:\Users\SEC\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:590: UserWarning: find_element_by_* commands are deprecated. Please use find_element() inst

In [22]:
baseball_data

[    순위  선수명   팀명    AVG  G  PA  AB   H 2B 3B HR RBI SB CS BB HBP SO GDP  E  \
 0    1  윤동균   OB  0.407  6  30  27  11  3  0  0   1  0  1  3   0  2   0  0   
 1    2  김유동   OB  0.400  6  28  25  10  0  0  3  12  0  1  2   0  3   0  0   
 2    3  함학수   삼성  0.333  6  26  21   7  2  0  1   4  0  1  3   1  2   0  2   
 3    4  정현발   삼성  0.300  6  23  20   6  0  1  1   2  0  0  3   0  2   0  0   
 4    5  배대웅   삼성  0.280  6  28  25   7  3  1  0   3  1  0  0   1  1   0  0   
 5    6  이만수   삼성  0.273  6  25  22   6  1  1  0   5  0  0  2   0  2   1  0   
 6    7  김우열   OB  0.250  6  30  28   7  0  0  2   6  0  0  2   0  1   0  0   
 7    8  구천서   OB  0.238  6  27  21   5  1  0  0   1  1  2  5   0  5   2  3   
 8    8  이홍범   OB  0.238  6  23  21   5  1  0  0   0  0  0  1   0  2   1  0   
 9   10  오대석   삼성  0.231  6  28  26   6  1  0  1   4  0  1  2   0  2   0  1   
 10  11  유지훤   OB  0.208  6  25  24   5  1  0  0   1  1  0  0   0  3   0  4   
 11  12  신경식   OB  0.190  6  29  21   4  3  0  0   3